# Umbrella Hook v4

## Key Facts

* Beam is 16mm w x 24mm tall at 12.5º angle
* Wing overhang 18mm-> 32mm

| Dimension   | value |
| ----------- | ----- |
| hook_length = 48
| beam_width  = 16
| beam_height = 24
| beam_angle  = 12.5


In [1]:
from math import sin,cos,tan, asin,acos, atan, radians, floor, degrees, sqrt
from datetime import datetime
from types import SimpleNamespace
from timeit import default_timer as timer

import pprint as pp
import cadquery as cq
from jupyter_cadquery import show, set_defaults, open_viewer, Camera
from jupyter_cadquery.replay import enable_replay, disable_replay, reset_replay, get_context, replay, Replay, _CTX

start = timer()
cv = open_viewer("Box", cad_width=780, height=525)
set_defaults(reset_camera=Camera.RESET, show_parent=False, axes=True, axes0=True)

use_replay = True
if use_replay:
    enable_replay(show_bbox=False, warning=False)
    reset_replay()
    show_object = replay
else:
    disable_replay()
    show_object = show

print(f"Initialized jupyter_cadquery with replay {datetime.now()}")
m = SimpleNamespace()
m.hook_length = 48	
m.beam_width = 16	
m.beam_height = 24	
m.beam_angle = 12.500	
m.drop = m.hook_length*sin(radians(m.beam_angle)) # positive
m.reach = m.hook_length*cos(radians(m.beam_angle))
m.body_thickness_far = 8
m.body_thickness_near = m.body_thickness_far + 0.5*m.drop
m.wing_thickness = 8
m.wing_drop_near = m.beam_height * 1.333
m.wing_drop_far = m.beam_height * 0.6667
m.wing_height_near = m.body_thickness_near + m.wing_drop_near
m.wing_height_far  = m.body_thickness_far + m.wing_drop_far
m.edge_fillet = 0.5
m.wing_drop = m.wing_drop_near - m.wing_drop_far
m.wing_hyp = sqrt(m.wing_drop**2 + m.reach**2)

m.h_ext = 12
m.scale_jaw = (m.reach - m.h_ext) / m.reach


m.jaw_drop = (m.wing_drop_near + m.drop - m.wing_drop_far)
m.stylish_angle = degrees(atan(m.jaw_drop / m.reach))


print(f"Building from model {m} in {timer() - start}")

Overwriting auto display for cadquery Workplane and Shape
Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList

Enabling jupyter_cadquery replay
Initialized jupyter_cadquery with replay 2025-11-17 17:52:05.858349
Building from model namespace(hook_length=48, beam_width=16, beam_height=24, beam_angle=12.5, drop=10.389101469028938, reach=46.8622083417568, body_thickness_far=8, body_thickness_near=13.194550734514468, wing_thickness=8, wing_drop_near=31.991999999999997, wing_drop_far=16.000799999999998, wing_height_near=45.18655073451447, wing_height_far=24.000799999999998, edge_fillet=0.5, wing_drop=15.9912, wing_hyp=49.515503108685266, h_ext=12, scale_jaw=0.7439301214213727, jaw_drop=26.380301469028936, stylish_angle=29.37661277344574) in 0.20979056000942364


In [2]:
# Make the top body
s = (
        cq.Workplane("YZ")
        .moveTo(0,m.body_thickness_near)
        # New
        .polarLine(m.body_thickness_near, -90)
        .polarLine(m.hook_length, m.beam_angle)
        .polarLine(m.body_thickness_far, 90)
        .close()

)
# box1 = cq.Workplane("YZ").placeSketch(s)
right_body = s.extrude(m.beam_width/2)

replay(right_body)

Use the multi select box below to select one or more steps you want to examine
+

In [3]:
# make bumps under body
bump = SimpleNamespace()
bump.height = 1.5
bump.width = 3
bump.gap = 4
bump.pitch = bump.width + bump.gap
bump.count = floor(m.hook_length / bump.pitch)
m.bump = bump

pp.pp(m)
right_body = (
    right_body.faces("<Z").workplane(centerOption="CenterOfMass")
        .rarray(0, m.bump.pitch, 1, m.bump.count, True)
        .box(m.beam_width/2,m.bump.width,m.bump.height)
)
replay(right_body)

namespace(hook_length=48,
          beam_width=16,
          beam_height=24,
          beam_angle=12.5,
          drop=10.389101469028938,
          reach=46.8622083417568,
          body_thickness_far=8,
          body_thickness_near=13.194550734514468,
          wing_thickness=8,
          wing_drop_near=31.991999999999997,
          wing_drop_far=16.000799999999998,
          wing_height_near=45.18655073451447,
          wing_height_far=24.000799999999998,
          edge_fillet=0.5,
          wing_drop=15.9912,
          wing_hyp=49.515503108685266,
          h_ext=12,
          scale_jaw=0.7439301214213727,
          jaw_drop=26.380301469028936,
          stylish_angle=29.37661277344574,
          bump=namespace(height=1.5, width=3, gap=4, pitch=7, count=6))
Use the multi select box below to select one or more steps you want to examine
+

In [4]:
# Make a wing
right_wing = (
    right_body.faces(">X").vertices("<YZ").workplane(centerOption="CenterOfMass")
    .moveTo(0,m.body_thickness_near)
    .polarLine(m.wing_height_near, -90) # down
    .line(m.reach, m.wing_drop_near + m.drop - m.wing_drop_far) # diag to far
    .polarLine(m.wing_height_far, 90) # up
    .close()

)

right_wing = (
    right_wing
    .extrude(m.wing_thickness)
    .edges(">(0, -1, 1)").fillet(m.edge_fillet) # top near
    .edges(">(0, 1, 1)").fillet(m.edge_fillet) # top far
    # .edges(">(1, 0, 1)").fillet(m.edge_fillet)
)
    
replay(right_wing)
# Mirror in YZ plane before finishing on right wing
left_wing = right_wing.mirror(mirrorPlane="YZ", basePointVector=(0, 0, -30))

Use the multi select box below to select one or more steps you want to examine
+

In [5]:
sig = SimpleNamespace()
sig.h_padding = 2
sig.v_padding = -0.5
sig.fontsize = 7
sig.depth=-1.5
sig.font = "Arial.ttf"
sig.fontface = 'bold'
sig.monogram = 'DFG'
m.sig = sig

signed_right_wing = (
    right_wing
        .faces(">X[-2]")
        .workplane()
        .center(-sig.h_padding ,-sig.v_padding)
        .transformed(offset=cq.Vector(0, -0, 0.0), rotate=cq.Vector(0, 0, -m.beam_angle))
        .text(f"{sig.monogram} {str(datetime.now().year)}", fontsize=sig.fontsize, distance=sig.depth, fontPath=sig.font,
         halign="right", valign="top", kind=sig.fontface)
)
signed_right_wing = (
    signed_right_wing
    .edges(">(0, -1, -1)").fillet(m.edge_fillet) # bottom near
    .edges(">(0, 1, -1)").fillet(m.edge_fillet) # bottom far
    .edges(">(1, -1, 0)").fillet(m.edge_fillet) # right face
)

replay(signed_right_wing)

Use the multi select box below to select one or more steps you want to examine
+

In [6]:
# make left wing from right+jaw


left_wing = (
    right_wing
    .mirror(mirrorPlane="YZ", basePointVector=(0, 0, -30))
    # select left face bottom corner
    .faces("<X").vertices("<Z").workplane(centerOption="CenterOfMass")
    # draw the jaw with line segments
    .line(-m.scale_jaw*m.reach, -m.scale_jaw*(m.wing_drop_near + m.drop - m.wing_drop_far)) # diag to far
    .polarLine(m.h_ext, -180) # jaw out
    .polarLine(m.wing_thickness, 90) # up    
    .polarLine(m.h_ext-2, 0) # jaw back
    .polarLine(0.45*m.scale_jaw*m.reach, m.stylish_angle)
    .polarLine(0.30*m.scale_jaw*m.reach, 55)
    .polarLine(0.05*m.scale_jaw*m.reach, 90)
    .polarLine(0.45*m.scale_jaw*m.reach, 125) # final leg long for design safety
    .polarLine(0.5*m.wing_height_far, 90)     # extra up turn to ensure closed against wing
    .close()
).extrude(-m.wing_thickness)

# fillet completed faces
left_wing = (
    left_wing
    .edges(">( -1, -0.2, 1)").fillet(m.edge_fillet) # left non-bottom edges
    .faces(">(0,-1, -1)").fillet(m.edge_fillet) # main jaw bottom
    .faces(">(0, 0, -1)").fillet(m.edge_fillet) # jaw ext bottom
    # .edges(">( 0, -1, -1)").fillet(m.edge_fillet) # near  bottom
    # .edges(">( 0,  1, -1)").fillet(m.edge_fillet) # far bottom
    # .edges(">(-1, -0, -1)").fillet(m.edge_fillet) # bottom-far-left (interfere tooth)
)

# build the tooth

tooth = SimpleNamespace()

tooth.height = 12
tooth.width = m.wing_thickness
tooth.length = m.wing_thickness
tooth.top_width = 2
tooth.top_length = 1
tooth.width_offset = 0 # l/r centering
tooth.length_offset = (m.h_ext - tooth.length) / 4
tooth.width_offset_top = 0 # l/r centering
tooth.length_offset_top = (tooth.top_length  - tooth.length) / 2
m.tooth = tooth

left_wing = (
    left_wing
    .faces("+Z").workplane(centerOption="CenterOfMass")
    .center(tooth.width_offset, tooth.length_offset)
    .rect(tooth.width, tooth.length)
    .workplane(offset=tooth.height)
    .center(tooth.width_offset_top, tooth.length_offset_top)
    .rect(tooth.top_width,tooth.top_length)
    .loft(combine=True)
)

replay(left_wing)

Use the multi select box below to select one or more steps you want to examine
+

In [7]:

body = signed_right_wing.union(left_wing)
replay(body)

Use the multi select box below to select one or more steps you want to examine
+

In [8]:

body.val().exportStl("Umbrella Hook v4.stl", ascii=True)

print(f"Built from model {pp.pp(m)} in {timer() - start}")

namespace(hook_length=48,
          beam_width=16,
          beam_height=24,
          beam_angle=12.5,
          drop=10.389101469028938,
          reach=46.8622083417568,
          body_thickness_far=8,
          body_thickness_near=13.194550734514468,
          wing_thickness=8,
          wing_drop_near=31.991999999999997,
          wing_drop_far=16.000799999999998,
          wing_height_near=45.18655073451447,
          wing_height_far=24.000799999999998,
          edge_fillet=0.5,
          wing_drop=15.9912,
          wing_hyp=49.515503108685266,
          h_ext=12,
          scale_jaw=0.7439301214213727,
          jaw_drop=26.380301469028936,
          stylish_angle=29.37661277344574,
          bump=namespace(height=1.5, width=3, gap=4, pitch=7, count=6),
          sig=namespace(h_padding=2,
                        v_padding=-0.5,
                        fontsize=7,
                        depth=-1.5,
                        font='Arial.ttf',
                        fontface='bo